## Cutting band structure or band mapping data to the first Brillouin zone

In [ ]:
import warnings as wn
wn.filterwarnings("ignore")

import numpy as np
import fuller
import scipy.io as sio
import matplotlib.pyplot as plt
%matplotlib inline

### 1. Load data

In [ ]:
bs_hse = sio.loadmat(r'../../data/theory/WSe2_HSE06_bands.mat')
bshse = np.moveaxis(bs_hse['evb'][::2,...], 1, 2)

In [ ]:
bs_hse['kxxsc'].shape

### 2. Determine Brillouin zone boundary using landmarks

In [ ]:
bzn = fuller.generator.BrillouinZoner(bands=bshse, axis=0)

In [ ]:
bzn.select_slice(selector=slice(0, 1))

In [ ]:
bzn.findlandmarks(image=bzn.slice, method='daofind', sigma=25, fwhm=40, sigma_radius=4, image_ofs=[25,25,0,0])

In [ ]:
bzn.visualize(bzn.slice, annotated=True, points=dict(pts=bzn.pouter_ord))
# plt.scatter(bzn.pcent[1], bzn.pcent[0], s=100, c='r')

### 3. Calculate geometric parameters for creating data mask

In [ ]:
imr, imc = bzn.slice.shape
hexside = np.linalg.norm(bzn.pouter_ord[1,:] - bzn.pouter_ord[2,:])
hexdiag = np.linalg.norm(bzn.pouter_ord[1,:] - bzn.pouter_ord[4,:])
imside = min(bzn.slice.shape)
ptop = (imr - hexdiag) / 2
# Calculate the distance of twice of the apothem (apo diameter)
apod = np.abs(np.sqrt(3)*hexside)
pleft = (imc - hexdiag) / 2

In [ ]:
bzn.maskgen(hexdiag=int(hexdiag), imside=imside, image=None, padded=True,
                               pad_top=int(ptop), pad_left=int(pleft), ret='all')

In [ ]:
bzn.visualize(bzn.mask*bzn.slice, annotated=True, points=dict(pts=bzn.pouter_ord))

In [ ]:
bzn.bandcutter(selector=slice(0, None))

Energy band data sliced to the first Brillouin zone now bears the name `bandcuts`

In [ ]:
bzn.bandcuts.shape

### 4. Save sliced data

In [ ]:
# Summarize the content of the class (remove the semiconlon to see output)
bzn.summary();

In [ ]:
# bzn.save_data(form='h5', save_addr=r'./wse2_hse_bandcuts.h5')

In [ ]:
plt.imshow(bzn.bandcuts[0,...])